In [172]:
import pandas as pd
df = pd.read_csv("/Users/iyeeun/Documents/coreview_3/data/df_sumWordSentence.csv")
df

,이름,작성일자,평점,리뷰 내용,맛 만족도,preprocessed_review,krwordrank
0,박*혜,2025.01.14,5,"짜파게티 마라, 어떤 제품일까?먼저 짜파게티 마라의 기본 정보를 알아보았어요. 이 ...",아주 넉넉해요,짜파게티 마라 어떤 제품 일까먼저 짜파게티 마라 기본 정보를 알아보았어요 제품 유명...,['마라']
1,dimss,2025.01.13,4,안녕하세요!! 리뷰하는 dimss입니다^^오늘도 솔직 사용후기 들고 왔어요❤️⬇️⬇...,적당히 남았어요,안녕하세요 리뷰 하는 입니다 오늘도 솔직 사용후기 들고 왔어요짜파게티 마라 솔직 후...,"['마라', '분들', '짜파']"
2,이*람,2025.01.12,5,### 구매 계기매운 음식을 좋아하는 저는 최근 마라탕이나 마라샹궈와 같은 마라 요...,적당히 남았어요,구매 계기매운 음식 좋아 하는 저는 최근 마라 탕이나 마라 샹궈와 같은 마라 요리 ...,"['마라', '매운', '맛이', '정말', '국물']"
3,시마에나가,2025.01.11,4,"⏹️ 요약 ⏹️✅ 구성 - 면, 후레이크, 짜장스프, 마라조미유✅ 맛 - 맵찔이 먹...",적당히 남았어요,요약 구성 후레이크 짜장스프 마라 조미유 맵찔이 먹으면 안됨 그냥 매운 마라 구매 ...,"['마라', '넣고', '짜파', '먹어']"
4,김*수,2025.01.11,5,짜파게티 마라를 구매한 후기를 이모티콘을 활용하여 작성해 보겠습니다. 아래와 같은 ...,아주 넉넉해요,짜파게티 마라 구매 후기 이모티콘을 활용 하여 작성해 보겠습니다 아래와 같은 방식 ...,"['마라', '매운', '맛이']"
...,...,...,...,...,...,...,...
259,잔잔다에,2025.01.01,1,처음 입에 넣을 때 거부감 드는 향과 뒤에 매운 맛만 느껴짐참고서라도 먹고싶지 않은 맛,맛 평가 없음,NaN,NaN
260,코만도x,2025.01.10,1,맛과 어울리지도 않게 맵기만함고급스러운 매운맛이 아니고 싼매운맛. 유통기한 끝나가는...,적당히 남았어요,NaN,NaN
261,sssgg,2025.01.04,1,마라향이 뭔가 조화롭지않아서 좀 인위적인?느낌의 향만나고우선 진짜 너무 맵기만해요,적당히 남았어요,NaN,NaN
262,쿠리사,2025.01.13,1,짜파게티도 맛있고 블랙도 맛있는데이건 영.....마라향이 확 나는게 아니라서 이상함...,아주 넉넉해요,NaN,NaN


In [175]:
import pandas as pd
import random
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# Load tokenizer and model for summarization
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')


# Function to generate n-grams from a text
def generate_ngrams(text, n=3):
    if pd.isna(text):
        return []
    words = text.split()
    ngrams = [' '.join(words[i:i+n]) for i in range(len(words) - n + 1)]
    return ngrams

# Function to extract n-grams containing a specific keyword
def extract_ngrams_with_keyword(data, keyword, review_column='preprocessed_review', n=3):
    # Generate n-grams for each review
    data['ngrams'] = data[review_column].apply(lambda x: generate_ngrams(x, n))
    
    # Filter n-grams containing the keyword
    data['filtered_ngrams'] = data['ngrams'].apply(lambda ngrams: [ngram for ngram in ngrams if keyword in ngram])
    return data

# Function to preprocess text before summarization
def preprocess_text(text):
    # Remove duplicate words and limit excessive repetition
    words = text.split()
    deduplicated = []
    for word in words:
        if len(deduplicated) < 2 or deduplicated[-2:] != [word, word]:
            deduplicated.append(word)
    return ' '.join(deduplicated)

# Function to summarize text using KoBART
def summarize_text_kobart(text, max_length=50, min_length=30):
    if not text or len(text.strip()) == 0:
        return None
    # Preprocess text to remove duplicates
    text = preprocess_text(text)
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs,
        max_length=max_length,
        min_length=min_length,
        length_penalty=1.0,
        num_beams=4,
        repetition_penalty=4.0,  # Higher penalty for repetitive outputs
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Main function to process and summarize the dataset
def process_and_summarize(df, keyword, n=5, samples_per_rating=3):
    # Extract n-grams with keyword
    df = extract_ngrams_with_keyword(df, keyword, review_column='리뷰 내용', n=n)

    # Create a new DataFrame to store results
    intermediate_results = []

    # Process each rating
    for rating in sorted(df['평점'].unique(), reverse=True):
        rating_df = df[df['평점'] == rating]
        all_ngrams = [ngram for ngrams in rating_df['filtered_ngrams'] for ngram in ngrams if ngram.strip() != ""]

        if not all_ngrams:  # Skip if no valid n-grams are found
            continue

        # Select the longest n-grams
        longest_ngrams = sorted(all_ngrams, key=len, reverse=True)[:samples_per_rating]

        # Summarize each longest n-gram
        summarized_ngrams = [summarize_text_kobart(ngram) for ngram in longest_ngrams]

        # Combine the summarized n-grams into one text
        combined_text = ' '.join(summarized_ngrams)

        # Summarize the combined text
        summary = summarize_text_kobart(combined_text, max_length=30, min_length=10)

        # Store intermediate results
        intermediate_results.append({
            '평점': rating,
            'summary': summary
        })

    # Create a final summary combining all summaries
    combined_summaries = ' '.join([result['summary'] for result in intermediate_results if result['summary']])
    final_summary = summarize_text_kobart(combined_summaries, max_length=30, min_length=10)

    # Append the final summary to the results
    intermediate_results.append({
        '평점': 'Final',
        'summary': final_summary
    })

    return pd.DataFrame(intermediate_results)

# Example usage
keyword = '짜파'
result_df = process_and_summarize(df, keyword)
print(result_df)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


      평점                                            summary
0      5             최애라면인 짜파게티에 마라가있다고하더라구요마라탕도좋아하는데 최애라면인
1      4       2025.12.27일유통기한:2025.06.19일 기본짜파게티의 고소하고풍미있는
2      3               ᄏᄏᄏ짜파게티에 대해 영향을줄 알았음 ᄏᄏᄏ짜파게티에 직접예약주문
3      2  짜파게티와 마라 냄새가 어우러지지 않으면 짜파게티와 마라 냄새가 생각보다 맵다고 마라볶음
4      1               마라샹궈에 굳이 양파도볶아서넣구 했는뎅뎅..마라탕이 짜찝찝한 매움
5  Final     2025.12.27일유통기한:2025.12.27일유통기한:2025.06.19일 기본


In [181]:
result_df['summary'][5]

'2025.12.27일유통기한:2025.12.27일유통기한:2025.06.19일 기본'

In [174]:
import pandas as pd
import random
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# Load tokenizer and model for summarization
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

# Function to generate n-grams around a keyword
def generate_ngrams_around_keyword(text, keyword, n=3):
    if pd.isna(text):
        return []
    words = text.split()
    ngrams = []
    for i, word in enumerate(words):
        if keyword in word:
            start_idx = max(0, i - n // 2)
            end_idx = min(len(words), i + n // 2 + 1)
            ngrams.append(" ".join(words[start_idx:end_idx]))
    return ngrams


# Function to generate n-grams from a text
def generate_ngrams(text, n=3):
    if pd.isna(text):
        return []
    words = text.split()
    ngrams = [' '.join(words[i:i+n]) for i in range(len(words) - n + 1)]
    return ngrams

# Function to extract n-grams containing a specific keyword
def extract_ngrams_with_keyword(data, keyword, review_column='preprocessed_review', n=3):
    # Generate n-grams for each review
    data['ngrams'] = data[review_column].apply(lambda x: generate_ngrams(x, n))
    
    # Filter n-grams containing the keyword
    data['filtered_ngrams'] = data['ngrams'].apply(lambda ngrams: [ngram for ngram in ngrams if keyword in ngram])
    
    # Select relevant columns for output
    result = data[[review_column, 'filtered_ngrams']]
    return result



# Function to summarize text using KoBART
def summarize_text_kobart(text, max_length=50, min_length=30):
    if not text or len(text.strip()) == 0:
        return None
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs,
        max_length=max_length,
        min_length=min_length,
        length_penalty=3.0,
        num_beams=4,
        repetition_penalty=2.0,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary
# Function to preprocess text before summarization
def preprocess_text(text):
    # Remove duplicate words and limit excessive repetition
    words = text.split()
    deduplicated = []
    for word in words:
        if len(deduplicated) < 2 or deduplicated[-2:] != [word, word]:
            deduplicated.append(word)
    return ' '.join(deduplicated)

# Updated function to summarize text using KoBART
def summarize_text_kobart(text, max_length=50, min_length=30):
    if not text or len(text.strip()) == 0:
        return None
    # Preprocess text to remove duplicates
    text = preprocess_text(text)
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs,
        max_length=max_length,
        min_length=min_length,
        length_penalty=1.0,
        num_beams=4,
        repetition_penalty=4.0,  # Higher penalty for repetitive outputs
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Main function to process and summarize the dataset
def process_and_summarize(df, keyword, n=5, samples_per_rating=3):
    # Generate n-grams for each review
    df['ngrams'] = df['리뷰 내용'].apply(lambda x: extract_ngrams_with_keyword(x, keyword, n))

    # Create a new DataFrame to store results
    intermediate_results = []

    # Process each rating
    for rating in sorted(df['평점'].unique(), reverse=True):
        rating_df = df[df['평점'] == rating]
        all_ngrams = [ngram for ngrams in rating_df['ngrams'] for ngram in ngrams if ngram.strip() != ""]

        if not all_ngrams:  # Skip if no valid n-grams are found
            continue

        # Select the longest n-grams
        longest_ngrams = sorted(all_ngrams, key=len, reverse=True)[:samples_per_rating]

        # Summarize each longest n-gram
        summarized_ngrams = [summarize_text_kobart(ngram) for ngram in longest_ngrams]

        # Combine the summarized n-grams into one text
        combined_text = ' '.join(summarized_ngrams)

        # Summarize the combined text
        summary = summarize_text_kobart(combined_text, max_length=30, min_length=10)

        # Store intermediate results
        intermediate_results.append({
            '평점': rating,
            'summary': summary
        })

    # Create a final summary combining all summaries
    combined_summaries = ' '.join([result['summary'] for result in intermediate_results if result['summary']])
    final_summary = summarize_text_kobart(combined_summaries, max_length=30, min_length=10)

    # Append the final summary to the results
    intermediate_results.append({
        '평점': 'Final',
        'summary': final_summary
    })

    return pd.DataFrame(intermediate_results)

keyword = '짜파'
result_df = process_and_summarize(df, keyword)
print(result_df)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


AttributeError: 'str' object has no attribute 'apply'

In [171]:
result_df['summary'][5]

'2025.06.19일쿠팡에서 짜장라면을 짜파파파파파파파파파파파파파'

In [37]:
def summarize_text_kobart(text, max_length=30, min_length=30):
    if not text or len(text.strip()) == 0:
        return None
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs,
        max_length=max_length,
        min_length=min_length,
        length_penalty=2.0,
        num_beams=4,
        repetition_penalty=2.0,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

print(summarize_text_kobart(result_df['mara_sentences'][0]))

짜파게티 마라는 중국 사천 지역의 매운맛과 향신료가 가미된 스타일로 짜파게티 마라를 조리하는 방법은 다른 즉석면과 크게 다르지 않아요


#### Ratings

In [22]:
import pandas as pd

def calculate_keyword_average_ratings(dataframe, keywords, review_column, rating_column):
    keyword_scores = {}
    for keyword in keywords:
        # Filter reviews that contain the keyword
        keyword_reviews = dataframe[dataframe[review_column].str.contains(keyword, na=False)]
        # Calculate the average rating
        avg_score = keyword_reviews[rating_column].mean()
        keyword_scores[keyword] = avg_score

    # Convert the results to a DataFrame
    result_df = pd.DataFrame(list(keyword_scores.items()), columns=['Keyword', 'Average Rating'])
    return result_df


keywords = ['마라', '짜파게티', '매운']  # keywordrank로 뽑아낸 k개 단어들의 리스트
result_df = calculate_keyword_average_ratings(df, keywords, 'preprocessed_review', '평점')
result_df

,Keyword,Average Rating
0,마라,4.570048
1,짜파게티,4.619048
2,매운,4.698276


In [8]:
import pandas as pd
import re

def analyze_keyword_reviews(data, keyword, review_column='preprocessed_review'):
    # Function to extract and summarize sentences containing the keyword
    def summarize_keyword_reviews(text):
        if pd.isna(text):
            return None
        sentences = re.split(r'[.!?]', text)  # Split into sentences
        keyword_sentences = [s.strip() for s in sentences if keyword in s]  # Filter sentences containing the keyword
        return keyword_sentences

    # Function to count occurrences of the keyword in a text
    def count_keyword_mentions(text):
        if pd.isna(text):
            return 0
        return text.count(keyword)

    # Extract keyword-related sentences
    data['keyword_sentences'] = data[review_column].apply(summarize_keyword_reviews)

    # Count keyword mentions in each review
    data['keyword_count'] = data[review_column].apply(count_keyword_mentions)

    # Sort the data by the number of keyword mentions, descending
    top_keyword_reviews = data.sort_values(by='keyword_count', ascending=False).head(5)

    # Reset index to ensure numerical order
    top_keyword_reviews = top_keyword_reviews.reset_index(drop=True)

    # Select relevant columns for output
    top_reviews = top_keyword_reviews[[review_column, 'keyword_count']]

    return top_reviews

# Example usage
review_result = analyze_keyword_reviews(df, keyword=word_result['Keyword'][2])
review_result


,preprocessed_review,keyword_count
0,짜파게티 마라 내돈내산으로 솔직 후기 입니다평소에 마라 탕을 엄청 좋아 하는 사람 ...,16
1,구매 이유 평소 짜파게티 즐겨 먹는데 최근 새로운 맛이 출시 되었다는 소식 듣고 궁...,15
2,마라 짜파게티 출시 됐다고 해서 쿠팡 에서 구입했 습니다사전 예약 구매 가능하다는 ...,14
3,구입 상품 짜파게티 마라 개원래 농심 짜파게티 사천짜파게티 즐겨 먹습니다 최근 에는...,14
4,농심 마라 짜파게티 개입 구입 후기에요일요일엔 내가 짜파게티 요리 사짜짜짜짜짜짜파게...,12


In [9]:
import pandas as pd
from itertools import islice

# Function to generate n-grams from a text
def generate_ngrams(text, n=3):
    if pd.isna(text):
        return []
    words = text.split()
    ngrams = [' '.join(words[i:i+n]) for i in range(len(words) - n + 1)]
    return ngrams

# Function to extract n-grams containing a specific keyword
def extract_ngrams_with_keyword(data, keyword, review_column='preprocessed_review', n=3):
    # Generate n-grams for each review
    data['ngrams'] = data[review_column].apply(lambda x: generate_ngrams(x, n))
    
    # Filter n-grams containing the keyword
    data['filtered_ngrams'] = data['ngrams'].apply(lambda ngrams: [ngram for ngram in ngrams if keyword in ngram])
    
    # Select relevant columns for output
    result = data[[review_column, 'filtered_ngrams']]
    return result

# Example usage
# Assuming your DataFrame is named `df` and you want to extract n-grams with the keyword '마라':
result = extract_ngrams_with_keyword(df, keyword=word_result['Keyword'][2], review_column='리뷰 내용', n=5)
result
# Save the result to a CSV file
# result.to_csv('ngram_results.csv', index=False, encoding='utf-8-sig')


,리뷰 내용,filtered_ngrams
0,"짜파게티 마라, 어떤 제품일까?먼저 짜파게티 마라의 기본 정보를 알아보았어요. 이 ...","[짜파게티 마라, 어떤 제품일까?먼저 짜파게티, 마라, 어떤 제품일까?먼저 짜파게티..."
1,안녕하세요!! 리뷰하는 dimss입니다^^오늘도 솔직 사용후기 들고 왔어요❤️⬇️⬇...,"[dimss입니다^^오늘도 솔직 사용후기 들고 왔어요❤️⬇️⬇️⬇️✨짜파게티, 솔직..."
2,### 구매 계기매운 음식을 좋아하는 저는 최근 마라탕이나 마라샹궈와 같은 마라 요...,"[빠져 있었습니다. 그러던 중 짜파게티의, 있었습니다. 그러던 중 짜파게티의 마라맛..."
3,"⏹️ 요약 ⏹️✅ 구성 - 면, 후레이크, 짜장스프, 마라조미유✅ 맛 - 맵찔이 먹...","[⏹️사전예약 중이길래 궁금해서 신청해보았어요.마침 짜파게티, 중이길래 궁금해서 신..."
4,짜파게티 마라를 구매한 후기를 이모티콘을 활용하여 작성해 보겠습니다. 아래와 같은 ...,"[짜파게티 마라를 구매한 후기를 이모티콘을, 방식으로 작성할 수 있습니다: 짜파게티..."
...,...,...
259,처음 입에 넣을 때 거부감 드는 향과 뒤에 매운 맛만 느껴짐참고서라도 먹고싶지 않은 맛,[]
260,맛과 어울리지도 않게 맵기만함고급스러운 매운맛이 아니고 싼매운맛. 유통기한 끝나가는...,[]
261,마라향이 뭔가 조화롭지않아서 좀 인위적인?느낌의 향만나고우선 진짜 너무 맵기만해요,[]
262,짜파게티도 맛있고 블랙도 맛있는데이건 영.....마라향이 확 나는게 아니라서 이상함...,[짜파게티도 맛있고 블랙도 맛있는데이건 영.....마라향이]
